In [4]:
#Colab: 04_train_transfer.ipynb
from google.colab import drive
drive.mount('/content/drive')
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.metrics import Precision, Recall
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pickle
PROJECT_ROOT='/content/drive/MyDrive/aerial_project'
CLASSIFICATION_DIR=os.path.join(PROJECT_ROOT, 'classification_dataset')
DETECTION_DIR=os.path.join(PROJECT_ROOT, 'object_detection_Dataset')
DATA_DIR=os.path.join(PROJECT_ROOT, 'data')
SAVED_MODELS=os.path.join(PROJECT_ROOT, 'saved_models')
RESULTS_DIR=os.path.join(PROJECT_ROOT, 'results')
#create important folders if they don't exist
os.makedirs(SAVED_MODELS, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
print('PROJECT_ROOT:', PROJECT_ROOT)
print('CLASSIFICATION_DIR exists:', os.path.exists(CLASSIFICATION_DIR))
print('DETECTION_DIR exists:', os.path.exists(DETECTION_DIR))

#Data augmentation setup (copied from nb. 02)
data_augmentation=tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.12),
    layers.RandomZoom(0.12),
    layers.RandomTranslation(0.08, 0.08),
], name='data_augmentation')

def build_efficientnet(input_shape=(224,224,3), num_classes=2, freeze_until=100):
  base=EfficientNetB0(include_top=False, input_shape=input_shape, weights='imagenet')
  base.trainable=True
  for layer in base.layers[:freeze_until]:
    layer.trainable=False

  inputs=layers.Input(shape=input_shape)
  x=layers.Rescaling(1./255)(inputs)
  x=data_augmentation(x)
  x=base(x, training=False)
  x=layers.GlobalAveragePooling2D()(x)
  x=layers.BatchNormalization()(x)
  x=layers.Dropout(0.4)(x)
  x=layers.Dense(256, activation='relu')(x)
  x=layers.Dropout(0.3)(x)
  outputs=layers.Dense(num_classes, activation='softmax')(x)
  return models.Model(inputs, outputs, name='efficientnet_b0')

model=build_efficientnet()
model.summary()

model.compile(
optimizer=tf.keras.optimizers.Adam(1e-4),
loss=tf.keras.losses.SparseCategoricalCrossentropy(),
metrics=['accuracy'])

callbacks=[
tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
tf.keras.callbacks.ModelCheckpoint(os.path.join(SAVED_MODELS,'best_efficientnet.h5'), save_best_only=True, monitor='val_loss'),
tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)]

#Dataset Loading and Class Weight Setup (copied from nb. 03)
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.utils import class_weight
import numpy as np

IMG_SIZE=(224, 224)
BATCH_SIZE=32

#train, val, test datasets
train_ds=tf.keras.utils.image_dataset_from_directory(
    os.path.join(CLASSIFICATION_DIR, 'train'),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

val_ds=tf.keras.utils.image_dataset_from_directory(
    os.path.join(CLASSIFICATION_DIR, 'valid'),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

test_ds=tf.keras.utils.image_dataset_from_directory(
    os.path.join(CLASSIFICATION_DIR, 'test'),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

# class names
class_names=train_ds.class_names
print("Classes:", class_names)

# Compute class weights (to handle imbalance)
labels=np.concatenate([y for x, y in train_ds], axis=0)
class_weights_vals=class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights=dict(enumerate(class_weights_vals))
print("Class Weights:", class_weights)

# Prefetch for performance
AUTOTUNE=tf.data.AUTOTUNE
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds=val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds=test_ds.cache().prefetch(buffer_size=AUTOTUNE)

history=model.fit(train_ds, validation_data=val_ds, epochs=30, callbacks=callbacks, class_weight=class_weights)

#optional fine-tuning: unfreezing the last N layers and train with lower lr
fine_tune_at=len(model.layers)-50
for layer in model.layers[fine_tune_at:]:
  layer.trainable=True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
loss=tf.keras.losses.SparseCategoricalCrossentropy(),
metrics=['accuracy'])
history_finetune=model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=callbacks, class_weight=class_weights)

#saving history and model
import pickle
with open(os.path.join(SAVED_MODELS,'history_efficientnet.pkl'),'wb') as f:
  pickle.dump({'initial': history.history, 'finetune': history_finetune.history}, f)
model.save(os.path.join(SAVED_MODELS,'best_efficientnet_full.h5'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PROJECT_ROOT: /content/drive/MyDrive/aerial_project
CLASSIFICATION_DIR exists: True
DETECTION_DIR exists: True
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "efficientnet_b0"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_2 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,383,141 (16.72 MB)

 Trainable params: 4,171,358 (15.91 MB)

...

Found 2662 files belonging to 2 classes.
Found 442 files belonging to 2 classes.
Found 215 files belonging to 2 classes.
Classes: ['bird', 'drone']
Class Weights: {0: np.float64(0.9413012729844413), 1: np.float64(1.0665064102564104)}
Epoch 1/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.6132 - loss: 1.0337

84/84 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.6134 - loss: 1.0328 - val_accuracy: 0.4910 - val_loss: 0.6988 - learning_rate: 1.0000e-04
Epoch 2/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy: 0.6689 - loss: 0.7779 - val_accuracy: 0.4910 - val_loss: 0.7572 - learning_rate: 1.0000e-04
Epoch 3/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 158ms/step - accuracy: 0.6614 - loss: 0.8366 - val_accuracy: 0.4932 - val_loss: 0.7912 - learning_rate: 1.0000e-04
Epoch 4/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.6770 - loss: 0.7488

84/84 ━━━━━━━━━━━━━━━━━━━━ 16s 185ms/step - accuracy: 0.6771 - loss: 0.7485 - val_accuracy: 0.7217 - val_loss: 0.5517 - learning_rate: 1.0000e-04
Epoch 5/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 162ms/step - accuracy: 0.7003 - loss: 0.6985 - val_accuracy: 0.6448 - val_loss: 0.6329 - learning_rate: 1.0000e-04
Epoch 6/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.7167 - loss: 0.6465

84/84 ━━━━━━━━━━━━━━━━━━━━ 16s 187ms/step - accuracy: 0.7165 - loss: 0.6471 - val_accuracy: 0.7896 - val_loss: 0.5313 - learning_rate: 1.0000e-04
Epoch 7/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 161ms/step - accuracy: 0.7283 - loss: 0.6276 - val_accuracy: 0.7805 - val_loss: 0.6223 - learning_rate: 1.0000e-04
Epoch 8/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 160ms/step - accuracy: 0.7165 - loss: 0.6003 - val_accuracy: 0.7104 - val_loss: 0.7109 - learning_rate: 1.0000e-04
Epoch 9/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.7340 - loss: 0.5567

84/84 ━━━━━━━━━━━━━━━━━━━━ 15s 184ms/step - accuracy: 0.7339 - loss: 0.5568 - val_accuracy: 0.8213 - val_loss: 0.4572 - learning_rate: 1.0000e-04
Epoch 10/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 162ms/step - accuracy: 0.7200 - loss: 0.5785 - val_accuracy: 0.7941 - val_loss: 0.5103 - learning_rate: 1.0000e-04
Epoch 11/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 20s 161ms/step - accuracy: 0.7192 - loss: 0.5639 - val_accuracy: 0.7919 - val_loss: 0.4799 - learning_rate: 1.0000e-04
Epoch 12/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 21s 161ms/step - accuracy: 0.7338 - loss: 0.5452 - val_accuracy: 0.7760 - val_loss: 0.4911 - learning_rate: 1.0000e-04
Epoch 13/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 161ms/step - accuracy: 0.7509 - loss: 0.5157 - val_accuracy: 0.7760 - val_loss: 0.5461 - learning_rate: 5.0000e-05
Epoch 14/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy: 0.7677 - loss: 0.4911 - val_accuracy: 0.8145 - val_loss: 0.5038 - learning_rate: 5.0000e-05
Epoch 15/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 163ms/step - accuracy

In [5]:
#Manual evaluation metrics (after training)
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
#Get predictions
y_true = np.concatenate([y for x, y in test_ds], axis=0)
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
#Classification report
print(classification_report(y_true, y_pred, target_names=class_names))
#Confusion matrix
cm=confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 508ms/step
              precision    recall  f1-score   support

        bird       0.88      0.69      0.77       121
       drone       0.69      0.87      0.77        94

    accuracy                           0.77       215
   macro avg       0.78      0.78      0.77       215
weighted avg       0.79      0.77      0.77       215

Confusion Matrix:
 [[84 37]
 [12 82]]
